In [2]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import transforms

import matplotlib.pyplot as plt
%matplotlib inline
#from PIL import Image # NOT SURE IF WE'RE USING THIS ONE

# NOT SURE WHAT THE POINT OF THE CODE BELOW IS

#if torch.cuda.is_available():
#    torch.backends.cuda.deterministic = True

## Load and Prepare Data

In [ ]:
# read data
df_train = pd.read_csv('log_train.csv')
df_test = pd.read_csv('log_test.csv')

# get data matrix and labels
train_data = df_train.iloc[:, 1:]
train_labels = df_train.iloc[:, 0]
test_data = df_test.iloc[:, 1:]
test_labels = df_test.iloc[:, 0]

data_dims = (19, 44)

In [ ]:
class logDataset(Dataset):
    """Log Anomaly Features Dataset"""
    
    def __init__(self, data_vec, dims, labels=None, transforms=None):
        self.X = data_vec
        self.dims = dims
        self.y = labels
        self.transforms = transforms
        
    def __len__(self):
        return len(self.X)
        
    def __getitem__(self, idx):
        data_matrix = self.X.iloc[idx, :]
        data_matrix = np.asarray(data_matrix).astype(np.float64).reshape(dims, 1)

        if self.transforms:
            data_matrix = self.transforms(data_matrix)
        
        if self.y:
            return(data_matrix, self.y[i])
        else:
            return data_matrix


In [ ]:
# NOTE THAT AS WE'VE DONE TF-IDF, WE MAY NOT WANT TO NORMALIZE
# HOWEVER, ONE OF THE ODD THINGS HERE IS THAT SINCE WE'RE PASSING FILTERS OVER THINGS
# IF THE COLUMNS REPRESENT SLIGHTLY DIFFERENT THINGS IT MAY BE AN ISSUE, ESPECIALLY FOR POOLING?
# NORMALLY EACH PIXEL REPRESENTS THE EXACT SAME THING, NOT SURE IF THAT'S STILL THE CASE AFTER
# TF-IDF

# define transforms
#train_mean = (0.5, 0.5, 0.5)
#train_std = (0.5, 0.5, 0.5)

apply_transforms = transforms.Compose([#transforms.Resize((32, 32)), # probably not using this
                                       transforms.ToTensor(),
                                       #transforms.Normalize(train_mean, train_std)])


In [ ]:
# use DataLoader

train_dataset = logDataset(train_data, dims = data_dims, transforms = apply_transforms)
test_dataset = logDataset(test_data, dims = data_dims, transforms = apply_transforms)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          num_workers=8,
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE,
                         num_workers=8,
                         shuffle=False)

# Checking the dataset
for data, labels in train_loader:  
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', data.shape)
    break

# Checking the dataset
for data, labels in train_loader:  
    print('Image batch dimensions:', data.shape)
    print('Image label dimensions:', data.shape)
    break

## Setup Model

In [ ]:
# Hyperparameters
RANDOM_SEED = 1
# THESE ARE ALL STARTING POINTS BASED ON LAB3
LEARNING_RATE = 0.001
BATCH_SIZE = 128
NUM_EPOCHS = 10

# Architecture
NUM_FEATURES = data_dims[0] * data_dims[1]
# not sure we should be using an architecture with 2 classes with softmax
# doesn't it make more sense to have a single output node
# previous labs/lectures have 2 classes with softmax which is somewhat confusing
NUM_CLASSES = 2

# Other
if torch.cuda.is_available():
    DEVICE = "cuda:0"
else:
    DEVICE = "cpu"


In [ ]:
# this is something to do with warming up the nn
# NEED TO FIGURE THIS OUT AND KEEP OR REMOVE
device = torch.device(DEVICE)
torch.manual_seed(0)

for epoch in range(2):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

In [ ]:
##########################
### MODEL
##########################


class logCNN(nn.Module):

    def __init__(self, num_classes):
        # removed super init here as not sure it's required
        self.num_classes = num_classes
        self.features = nn.Sequential(
            
            # Selection of 16 and 32 based on discussion with Debangsha, use for now
            # will need to figure out kernel size that works for dims and potentially add pooling
            # need to figure out how to deal with non-square image
            nn.Conv2d(1, 16, kernel_size=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 32, kernel_size=2),
            nn.ReLU()
            #nn.MaxPool2d(kernel_size=3) # maybe not using the last pooling layer as the size may be quite small
        )

        self.classifier = nn.Sequential(
            # NEED TO FIGURE OUT THE SIZE OF THE IMAGE FOR THE *4*4 part
            nn.Linear(32*4*4, 120), # NEED TO DECIDE ON NUMBER OF NODES (120 BASED ON LAB3)
            nn.ReLU(),
            nn.Linear(120, 84), # NEED TO DECIDE ON NUMBER OF NODES (84 BASED ON LAB3)
            nn.ReLU(),
            nn.Linear(84, num_classes), # NOT SURE WE WANT TO BE USING 2 NODE OUTPUT
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1) # NOT SURE WE NEED SOFTMAX
        return logits, probas

In [ ]:
torch.manual_seed(RANDOM_SEED)

model = logCNN(NUM_CLASSES) # THIS WILL BE UPDATED IF NOT USING 2 OUTPUT NODES
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train Model

In [ ]:
# NEED TO FIGURE OUT THE BEST FUNCTION FOR ACCURACY
# NEED TO FOCUS ON TRUE/FALSE
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100
    

start_time = time.time()
for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets) # NEED TO MODIFY BASED ON BEST COST FUNCTION
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING - MAY WANT TO MODIFY LOGGING INTERVALS
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))

        

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE)))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

## Evaluation

In [ ]:
with torch.set_grad_enabled(False): # save memory during inference
    # THIS NEEDS TO BE UPDATED BASED ON THE FUNCTION NAME WE'RE USING
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

In [1]:
# WE NEED TO BE EXTENDING THE PERFORMANCE MEASURES TO INCLUDE THOSE USED IN THE PAPERS
# F SCORE, PRECISION ETC ETC, MOST OF THE PAPERS SEEM TO USE SIMILAR METRICS, SHOULD BE RELATIVELY SIMPLE
# AND COULD BE POTENTIALLY BUILD DIRECTLY INTO THE COST FUNCTION (THE FUNCTION COULD RETURN MULTIPLE PARAMS AND WE WOULD OBVIOUSLY JUST USE THE ONE WE NEEDED FOR MODEL TRAINING BUT THEN WE WOULD HAVE ACCESS TO ALL OF THEM INSTEAD OF WRITING ANOTHER FUNCTION)

In [2]:
# ALSO RECOMMEND SOME SORT OF VIS TOOL ON BEING ABLE TO LOOK AT A COUPLE OF THE TIME IMAGES THAT ARE LABELLED ANOMALOUS CORRECTLY AND ONES THAT ARE LABELLED ANOMLAOUS INCORRECTLY OR NORMAL INCORRECTLY

# MAY REQURIE PASSING THE THE TIME IMAGE TO PIL (IF THIS DIDN"T NEED TO BE ALREADY DONE ABOVE)